In [1]:
%%capture
!pip install -q langchain==0.0.208 chromadb openai==0.27.8 python-dotenv yt_dlp
!pip install -q git+https://github.com/openai/whisper.git

In [2]:
import os

from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
import textwrap

import yt_dlp
import whisper
from langchain import OpenAI, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [4]:
def download_mp4_from_youtube(url):
    # Set the options for the download
    filename = 'lecuninterview.mp4'
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }

    # Download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
download_mp4_from_youtube(url)

In [5]:
model = whisper.load_model('base')
result = model.transcribe("lecuninterview.mp4")
print(result['text'])

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 35.1MiB/s]


 Hi, I'm Craig Smith and this is I on A On. This week I talked to Jan LeCoon, one of the seminal figures in deep learning development and a long time proponent of self-supervised learning. Jan spoke about what's missing in large language models and about his new joint embedding predictive architecture which may be a step toward filling that gap. He also talked about his theory of consciousness and the potential for AI systems to someday exhibit the features of consciousness. It's a fascinating conversation that I hope you'll enjoy. Okay, so Jan, it's great to see you again. I wanted to talk to you about where you've gone with so supervised learning since last week spoke. In particular, I'm interested in how it relates to large language models because the large language models really came on stream since we spoke. In fact, in your talk about JEPA, which is joint embedding predictive architecture. There you go. Thank you. You mentioned that large language models lack a world model. I wan

In [6]:
with open("interview.txt",'w') as file:
  file.write(result["text"])

In [11]:
##Load the model, split the text and then summarize it
llm = OpenAI(model_name='text-davinci-003',temperature=0)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    #seperators=[" ",",","\n"]
)

with open("interview.txt") as f:
  text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in text[:4]]  ##only using first four blocks since the context length is just 256

In [12]:
prompt_template = """Write a concise bullet point summary of the following:

{text}

CONSCISE SUMMARY IN BULLET POINTS:
"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                                     input_variables=["text"])

chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary,
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)


• Humanitarian aid is assistance given to people in need
• It can be provided in the form of food, shelter, medical care, and other essential services
• Humanitarian aid is often provided in response to natural disasters, conflicts, and other crises
• It is typically provided by governments, international organizations, and non-governmental organizations
